# 2020/03/10 공적마스크 재고 현황 API 주소 기반 사용 도구

* 이 코드는 [제](https://github.com/combacsa) 개인 편의를 위해 작성한 것을 지인들과 나누는 것입니다. 
* 언제든 사용 불가능하게 될 수 있으니 조심하세요. 또한 정상 작동을 전혀 보장하지 않습니다.
* 사용 전 [한국정보화진흥원 공공데이터활용지원센터의 가이드라인](https://www.data.go.kr/information/NOTICE_0000000001620/notice.do/)을 반드시 먼저 읽어보시기 바라며, 특히 아래 내용을 꼭 유념해 주세요.
  * 이번 주는 서비스의 안정적 운영을 지켜보기 위하여 베타서비스라고 합니다.
  * 서비스되는 재고 현황 정보는 데이터 처리 및 전송으로 인해 실제 현장 판매처의 현황과 5분~10분 정도의 차이가 있습니다.
  * 일부 약국에서는 번호표 배부 후 판매하는 약국도 있어 서비스되는 정보가 번호표 배부 현황을 반영하지는 못합니다.
  * 마스크 현황 정보는 성인용 마스크를 대상으로 합니다. 
  * 서비스 이용 가능 시간은 오전 8시부터 오후 11시까지입니다.
  * [식약처 공적마스크 구매 안내글](http://blog.naver.com/kfdazzang/221839489769)도 읽어 주세요. 
  * 어려운 환경에서도 일선에서 공헌해 주시는 약사님, 우체국 종사자분, 그리고 아마도 하나로 마트 분들께도 감사드립니다.
* 실제 호출하는 API는 [명세](https://app.swaggerhub.com/apis-docs/Promptech/public-mask-info/20200307-oas3#/)를 참고해 주세요.

# 여기 코드가 숨어 있어요. 굳이 펼쳐보지 않아도 괜찮아요.

In [0]:
import collections

import requests

API_URL = 'https://8oi9s0nnth.apigw.ntruss.com/corona19-masks/v1'
ACTION = 'storesByAddr/json'

TYPE = {'01': '약국', '02': '우체국', '03': '농협'}
COLOR = {'plenty': '초록색', 'some': '노란색', 'few': '빨간색', 'empty': '회색'}

def query(address, skip_red=True):
    params = {'address': address}
    try:
        result = requests.get(f'{API_URL}/{ACTION}', params=params,
                              timeout=10.0)
    except requests.exceptions.Timeout:
        print('이런, API 서버가 10초동안 응답이 없습니다. 다른 방법을 찾아봅시다. ㅠ_ㅠ')
        return
    if result.status_code != 200:
        print('이런, status_code 가 200 이 아닙니다. 뭔가 문제가 생겼나봅니다. 디버깅합시다.')
        print(result.status_code)
        print(result.content)
        return 

    data = result.json()  # JSON 포맷으로 되어 있는 자료를 받아옵니다.
    print(f'입력한 주소 "{address}"에서 {data["count"]}개 장소를 찾았습니다.')
    if data['count'] < 1:
        print('한 곳도 없는데, 주소를 "서울특별시 성동구 성수동"처럼 입력해주세요.')
        print('어쩌면 법정동 등의 문제일지도 몰라요. 서울특별시 대학동 -> 신림동으로 바꿔보세요.')
        print(repr(data))
        return
    
    stores_by_stat = collections.defaultdict(list)
    for store in data['stores']:
        stores_by_stat[store['remain_stat']].append(store)

    len_by_stat = collections.defaultdict(int)
    for stat, stores in stores_by_stat.items():
        len_by_stat[stat] = len(stores)

    green = len_by_stat['plenty']
    yellow = len_by_stat['some']
    red = len_by_stat['few']
    gray = len_by_stat['empty']
    error = sum(len_by_stat.values()) - green - yellow - red - gray

    def _print_store(idx, store):
        category = TYPE[store['type']]
        color = COLOR[store['remain_stat']]
        print(f'이름: {store["name"]} ({category} / {color})')
        print(f'주소: {store["addr"]}')
        print(f'입고: {store["stock_at"]} / 데이터 생성시점: {store["created_at"]}')
        print()

    if green:
        print(f'초록색(100개 이상)인 곳: [{green}]')
        print()
        for idx, store in enumerate(stores_by_stat['plenty']):
            _print_store(idx + 1, store)
        print()
    if yellow:
        print(f'노란색(30개 이상 100개 미만)인 곳: [{yellow}]')
        print()
        for idx, store in enumerate(stores_by_stat['some']):
            _print_store(idx + 1, store)
        print()
    
    if not skip_red:
        if red:
            print(f'빨간색(2개 이상 30개 미만)인 곳: [{red}]')
            print()
            for idx, store in enumerate(stores_by_stat['few']):
                _print_store(idx + 1, store)
            print()
        if gray:
            print(f'회색(사실상 마스크가 다 떨어진)인 곳: [{gray}]')
            print()
            for idx, store in enumerate(stores_by_stat['gray']):
                _print_store(idx + 1, store)
            print()
    if error:
        print(f'에러(API 결과가 이상함)인 곳: [{error}]')
    if skip_red and not green and not yellow:
        print('이런!! 다들 재고가 없네요. ㅠㅠ')

# 공적마스크 재고 현황 API 중 주소 기반 재고검색 결과를 활용하기

* 이 코드는 어떠한 개인정보도 수집하지 않습니다. 코드를 실행하였을 때 저장되는 정보는 전부 잠시동안만 Google CoLab Runtime 이라는 것에 저장됩니다. 만일 제 실수로 이 코드가 개인정보를 수집하고 있다면 꼭 [제](https://github.com/combacsa)게 알려주세요. 
* 아래 코드의 삼각형 모양을 눌러서, "오른쪽 흰 박스에 주소를 입력하고 엔터!" 라는 말이 뜨면, 거기에 찾고자 하는 주소를 광역시/도 시/군/구까지, 혹은 읍/면/동까지 입력해주세요.
  * 예시 1: 서울특별시 성동구 성동구 성수동1가
  * 예시 2: 경기도 부천시 신중동
  

In [0]:
# 화면 왼쪽의 |> 모양 아이콘을 누르면 주소를 물어봅니다. 찾아보고자 하는 주소를 입력해 주세요.
# |> 대신 [30]같은 숫자만 보이시나요? 숫자 위에 마우스를 올려놓으면 |> 로 바뀝니다.
# eg) 서울특별시 성동구 성수동 처럼, 광역시/도 구시군 혹은 읍면동까지 입력해주시는 게 제일 좋습니다.
# 행정동으로는 검색이 안 되고, 법정동으로만 검색이 되는 점 참고해주세요.
query_address = input('주소를 입력하고 엔터!')
query(query_address)